# Meta-data overview

In [1]:
from fdsobj import FdsObj
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

datadir = "raw/"

In [2]:
fname = '058.5_Weight_Car2_5_4_3_2_1_passenger_60kmh_West_And_East_2020.02.26.03.27.25.298014_20210201T185138_OPD.fds'
fname = os.path.join(datadir, fname)

fds = FdsObj(fname)
fds

FDS File: 058.5_Weight_Car2_5_4_3_2_1_passenger_60kmh_West_And_East_2020.02.26.03.27.25.298014_20210201T185138_OPD.fds
data_type: real32
start_bin: 0
end_bin: 1027
start_shot: 0 
sample_rate_Hz: 1000.04
bin_spacing_m: 0.68063
block_size: 1

In [3]:
def reload_data(fname, numShots):
    fds = FdsObj(fname)
    data = np.asarray([shot for _,shot in zip(range(numShots), fds)])
    data = np.transpose(data)
    return fds, data #datashape: (bin, shot)

In [4]:
numShots = 10000000
fds, data = reload_data(fname, numShots)

In [5]:
print(data.shape)

(1028, 515456)


# generate data

In [6]:
speed = "058.5"
numShots = 515456
bins = np.array([300,350,410,460,500,550,600,650,703,750])
start_bin, end_bin = bins[0], bins[-1] # we only manually labelled bins between start_bin and end_bin 
fname = 'raw/058.5_Weight_Car2_5_4_3_2_1_passenger_60kmh_West_And_East_2020.02.26.03.27.25.298014_20210201T185138_OPD.fds'
directions  = ["west", "east"]
clean_cars = ["1", "3", "4"]

In [7]:
from utils.data_preprocessing import *

In [8]:
fds, data = reload_data(fname, numShots)
output_labels = np.zeros((data.shape[0], data.shape[1])) #create placeholder output_labels
get_all_cars_window_width = []

## (1) label "clean" cars: DAS signals in these car classes all can be defined by human eyes.

In [9]:
for car_label in clean_cars:
    print("="*40)
    print("Process {0} car".format(car_label))
    for d in directions:
        print("Direction {0}".format(d))
        shot_start, shot_end = get_start_end_shot(str(car_label), speed, d)
        a1, b1, a2, b2 = get_a1b1_a2b2(bins, shot_start, shot_end) # get two LR slopes
        check_slope(a1, a2, car_label, d)
        output_labels, window_width_list = get_labels(output_labels, start_bin, end_bin, a1, b1, a2, b2, car_label)
        get_all_cars_window_width.extend(window_width_list)
    print("Done")
    #if denoise_method = False, output data will contain only raw windows. 

Process 1 car
Direction west
Direction east
Done
Process 3 car
Direction west
Direction east
Done
Process 4 car
Direction west
Direction east
Done


## (2) label "unclean" cars

In [10]:
### label car 2 

In [11]:
car_label = 2
print("="*40)
print("Process {0} car".format(car_label))
d = "west"
print("Direction {0}".format(d))

bins_clean, shot_start, shot_end = get_start_end_shot(str(car_label), speed, d)
a1, b1, a2, b2 = get_a1b1_a2b2(bins_clean, shot_start, shot_end) # get slopes
check_slope(a1, a2, car_label, d)
output_labels, window_width_list = get_labels(output_labels, start_bin, end_bin, a1, b1, a2, b2, car_label, 520, 570)
get_all_cars_window_width.extend(window_width_list)


d = "east"
print("Direction {0}".format(d))
shot_start, shot_end = get_start_end_shot(str(car_label), speed, d)
a1, b1, a2, b2 = get_a1b1_a2b2(bins, shot_start, shot_end) # get two LR slopes
check_slope(a1, a2, car_label, d)
output_labels, window_width_list = get_labels(output_labels, start_bin, end_bin, a1, b1, a2, b2, car_label)
get_all_cars_window_width.extend(window_width_list)
print("Done")    

Process 2 car
Direction west
Direction east
Done


In [12]:
### label car 5

In [13]:
car_label = 5
print("="*40)
print("Process {0} car".format(car_label))
d = "west"
print("Direction {0}".format(d))
shot_start, shot_end = get_start_end_shot(str(car_label), speed, d)
a1, b1, a2, b2 = get_a1b1_a2b2(bins, shot_start, shot_end) # get slopes
check_slope(a1, a2, car_label, d)
output_labels, window_width_list = get_labels(output_labels, start_bin, end_bin, a1, b1, a2, b2, car_label)
get_all_cars_window_width.extend(window_width_list)

d = "east"
print("Direction {0}".format(d))
bins_clean, shot_start, shot_end = get_start_end_shot(str(car_label), speed, d)
a1, b1, a2, b2 = get_a1b1_a2b2(bins_clean, shot_start, shot_end) # get slopes
check_slope(a1, a2, car_label, d)
output_labels, window_width_list = get_labels(output_labels, start_bin, end_bin, a1, b1, a2, b2, car_label, 300, 420)
get_all_cars_window_width.extend(window_width_list)
print("Done")   

Process 5 car
Direction west
Direction east
Done


In [14]:
import os 
save_root_folder = 'preprocessed_data/'
if not os.path.exists(save_root_folder):
    os.mkdir(save_root_folder)

output_fname_x, output_fname_y = save_root_folder+'058_5p_to_1p_X.txt', save_root_folder+'058_5p_to_1p_y.txt'
car_unique_labels = [1, 2, 3, 4, 5]
biggest_window_len = 5976 # we will pad all signals to this length
export_data_v2(data, output_labels, start_bin, end_bin, biggest_window_len, output_fname_x, output_fname_y)
